# Temperature and Immigration Data ETL

#### Project Summary
To build an ETL pipeline with Spark using data that Udacity had provided on Immigration and Temperature patterns in US cities. The data gets transformed in a relational data model that is optimized to analyse immigration events. The database then can be used to understand immigration patterns with respect to temperature in US cities

The project follows the follow steps:
* Step 1: Scope the Project and Gather Data
* Step 2: Explore and Assess the Data
* Step 3: Define the Data Model
* Step 4: Run ETL to Model the Data
* Step 5: Complete Project Write Up

## Importing the libraries

In [1]:
import pandas as pd
from datetime import datetime

### Step 1: Scope the Project and Gather Data

#### Scope 

The scope of the project is to create two dimensions table and one fact table. The first dimension table would be for the Immigration data and the second dimension table would be for Temperature Data. The fact table will be formed by joining on Immigration and Temperature. We will be using Spark to process the data.

#### Describe and Gather Data
The I94 immigration data comes from the US National Tourism and Trade Office website. It is provided in SAS7BDAT format which is a binary database storage format.

####  Immigration Data
This data comes from the US National Tourism and Trade Office. A data dictionary is included in the workspace. The immigration data has the following columns and I have taken from 
the knowledge section what each of them mean
- cicid >>>> CICID is a unique number for the immigrants. (No null values found).
- I94res>>>> is country from where one has travelled. (No null values found).
- I94addr>>>> is where the immigrants resides in USA . (Found null values).
- arrdate>>>> is date of arrrival . (Convert it to timestamp format).
- visatype>>>> is the type of visa which one owns . (No null values found).
- i94yr>>>>4 digit year
- i94mon>>>>Numeric month
- i94cit>>>> 3 digit code of origin city
- i94port>>>> 3 character code of destination USA city
- i94mode>>>> 1 digit travel code (transportation)
- FLTNO >>>> Flight number of Airline used to arrive in U.S
- VISATYPE >>>> Class of admission legally admitting the non-immigrant to temporarily stay in U.S.
- ADMNUM>>> Admission Number
- AIRLINE >>>> Airline used to arrive in U.S
- BIRYEAR>>>> 4 digit year of birth
- I94BIR>>>>> Age of Respondent in Years
- DEPDATE>>>>>> the Departure Date from the USA.
 
  

In [2]:
# Read in the data here (only for the month of April)
immigration_data = '../../data/18-83510-I94-Data-2016/i94_apr16_sub.sas7bdat'
df_immi = pd.read_sas(immigration_data, 'sas7bdat', encoding="ISO-8859-1")

In [3]:
len(df_immi)

3096313

In [4]:
df_immi.columns

Index(['cicid', 'i94yr', 'i94mon', 'i94cit', 'i94res', 'i94port', 'arrdate',
       'i94mode', 'i94addr', 'depdate', 'i94bir', 'i94visa', 'count',
       'dtadfile', 'visapost', 'occup', 'entdepa', 'entdepd', 'entdepu',
       'matflag', 'biryear', 'dtaddto', 'gender', 'insnum', 'airline',
       'admnum', 'fltno', 'visatype'],
      dtype='object')

In [5]:
df_immi.head(5)

,cicid,i94yr,i94mon,i94cit,i94res,i94port,arrdate,i94mode,i94addr,depdate,...,entdepu,matflag,biryear,dtaddto,gender,insnum,airline,admnum,fltno,visatype
0,6.0,2016.0,4.0,692.0,692.0,XXX,20573.0,NaN,NaN,NaN,...,U,NaN,1979.0,10282016,NaN,NaN,NaN,1.897628e+09,NaN,B2
1,7.0,2016.0,4.0,254.0,276.0,ATL,20551.0,1.0,AL,NaN,...,Y,NaN,1991.0,D/S,M,NaN,NaN,3.736796e+09,00296,F1
2,15.0,2016.0,4.0,101.0,101.0,WAS,20545.0,1.0,MI,20691.0,...,NaN,M,1961.0,09302016,M,NaN,OS,6.666432e+08,93,B2
3,16.0,2016.0,4.0,101.0,101.0,NYC,20545.0,1.0,MA,20567.0,...,NaN,M,1988.0,09302016,NaN,NaN,AA,9.246846e+10,00199,B2
4,17.0,2016.0,4.0,101.0,101.0,NYC,20545.0,1.0,MA,20567.0,...,NaN,M,2012.0,09302016,NaN,NaN,AA,9.246846e+10,00199,B2


#### Temperature Data 

The temperature data comes from Kaggle 
https://www.kaggle.com/berkeleyearth/climate-change-earth-surface-temperature-data 

The data comes from Kaggle.
- dt: Date
- AverageTemperature: Temperature data
- Average Temperature Uncertainity: Uncertainity in the data
- City
- Country
- Latitude
- Longitude

In [28]:
## Reading in the temperature data
temp_data = '../../data2/GlobalLandTemperaturesByCity.csv'
df_temp = pd.read_csv(temp_data, sep = ',')

In [14]:
df_temp.head(5)

,dt,AverageTemperature,AverageTemperatureUncertainty,City,Country,Latitude,Longitude
0,1743-11-01,6.068,1.737,Århus,Denmark,57.05N,10.33E
1,1743-12-01,NaN,NaN,Århus,Denmark,57.05N,10.33E
2,1744-01-01,NaN,NaN,Århus,Denmark,57.05N,10.33E
3,1744-02-01,NaN,NaN,Århus,Denmark,57.05N,10.33E
4,1744-03-01,NaN,NaN,Århus,Denmark,57.05N,10.33E


In [9]:
len(df_temp)

8599212

In [10]:
df_temp.tail(10)


,dt,AverageTemperature,AverageTemperatureUncertainty,City,Country,Latitude,Longitude
8599202,2012-12-01,4.303,0.341,Zwolle,Netherlands,52.24N,5.26E
8599203,2013-01-01,1.479,0.217,Zwolle,Netherlands,52.24N,5.26E
8599204,2013-02-01,1.559,0.304,Zwolle,Netherlands,52.24N,5.26E
8599205,2013-03-01,2.253,0.267,Zwolle,Netherlands,52.24N,5.26E
8599206,2013-04-01,7.710,0.182,Zwolle,Netherlands,52.24N,5.26E
8599207,2013-05-01,11.464,0.236,Zwolle,Netherlands,52.24N,5.26E
8599208,2013-06-01,15.043,0.261,Zwolle,Netherlands,52.24N,5.26E
8599209,2013-07-01,18.775,0.193,Zwolle,Netherlands,52.24N,5.26E
8599210,2013-08-01,18.025,0.298,Zwolle,Netherlands,52.24N,5.26E
8599211,2013-09-01,NaN,NaN,Zwolle,Netherlands,52.24N,5.26E


## Creating a Spark session

In [3]:
	
## Create spark session 
from pyspark.sql import SparkSession
spark = SparkSession.builder.\
config("spark.jars.packages","saurfang:spark-sas7bdat:2.0.0-s_2.11")\
.enableHiveSupport().getOrCreate()


### Step 2: Explore and Assess the Data
#### Explore the Data 
I started with destination ports and cleaned them up. I understand that there are other columns that have missing values or the date column or Nan values in the gender,adress,matflag and so on. Since my analysis has to do with more destination ports and their temperature dependence, I would not be deleting rows that have Nan in them specially in columns that I have interest. I shall however look into destination ports column and the residential adress column

#### Cleaning Steps
Document steps necessary to clean the data

In [10]:
df_immi.columns[df_immi.isna().any()].tolist()

['i94mode',
 'i94addr',
 'depdate',
 'i94bir',
 'dtadfile',
 'visapost',
 'occup',
 'entdepa',
 'entdepd',
 'entdepu',
 'matflag',
 'biryear',
 'dtaddto',
 'gender',
 'insnum',
 'airline',
 'fltno']

In [11]:
### 3522 rows that doesnt have the destination port. It wouldnt help us with our data exploration
len(df_immi[df_immi['i94port'] == 'XXX'])

3522

In [12]:
## Okay so these ports dont exist as well and there are 4215 rows
ports_that_dont_exist = ['FRG','HRL','ISP','JSJ','BUS','IAG','PHN','STN','VMB','T01','PHF','DRV','FTB','GAC','GMT','JFA','JMZ','NC8','NYL','OAI','PCW','WA5','WTR', 'X96', 'XNA',
'YGF','5T6', '060','SP0', 'W55','X44', 'AUH','RYY','SUS','74S','ATW','CPX','MTH','PFN','SCH','ASI', 'BKF', 'DAY', 'Y62' 'AG','BCM','DEC','PLB', 'CXO', 'JBQ', 'JIG',
'OGS','TIW','OTS','AMT','EGE','GPI','NGL','OLM','.GA','CLX', 'CP ', 'FSC', 'NK',  'ADU','AKT','LIT', 'A2A','OSN']
df_immi['i94port'].isin(ports_that_dont_exist).sum()

4215

In [4]:
## Now i94_port has all the ports that are valid. Therefore the dictionary has all the valid ports. Used a regex format to get all such columns. The 
## explanation for regex is given below
import re
re_obj = re.compile(r'\'(.*)\'.*\'(.*)\'')
i94_port = {}
with open('i94_port.txt') as f:
     for data in f:
             match = re_obj.search(data)
             i94_port[match[1]]=[match[2]]

- \': This gets the "'" from the text
- (.*)" : This gives us the first group upto and untill the end of the letters
- \' : This gives us the next "'" from the text
- .* : This gives us everything after until the next "'"
- (.*) : This gives us the next group. i.e: The second match
- \' : This gives us the last '

In [5]:
## Creating a dictionary for the residential address using regex
re_obj = re.compile(r'\s*(\d.*\d)\s.*\s(\'\w.*)')
i94_res_add = {}
with open('i94_res_cit.txt') as f:
     for data in f:
           # print(data)
            match = re_obj.search(data)
            i94_res_add[match[1]]=[match[2]]
           

- \s*: Every white space untill group 1
- (\d.*\d): First group and it has all the digits
- \s.*\s: The next bunch of white space
- (\'\w.*): The second group that has the name of the city

In [23]:
## Creating a function to clean the immigration data
def clean_immi(data,dictionary,col):
    '''
    Input
    Data: The input file for cleaning
    Dictionary: The dictionary that has the valid key-values combinations
    '''
    ## Read the file into spark
    df_immi = spark.read.format('com.github.saurfang.sas.spark').load(data)
    
    ## Cleaning the invalid codes
    df_immi = df_immi.filter(df_immi[col].isin(list(dictionary.keys())))
    
    return df_immi 

In [24]:
## This testing if the function works
start=datetime.now()
immi_test = '../../data/18-83510-I94-Data-2016/i94_apr16_sub.sas7bdat' 
df_immi_c= clean_immi(immi_test,i94_port,"i94port")
df_immi_c.show()
print (datetime.now()-start)

+-----+------+------+------+------+-------+-------+-------+-------+-------+------+-------+-----+--------+--------+-----+-------+-------+-------+-------+-------+--------+------+------+-------+--------------+-----+--------+
|cicid| i94yr|i94mon|i94cit|i94res|i94port|arrdate|i94mode|i94addr|depdate|i94bir|i94visa|count|dtadfile|visapost|occup|entdepa|entdepd|entdepu|matflag|biryear| dtaddto|gender|insnum|airline|        admnum|fltno|visatype|
+-----+------+------+------+------+-------+-------+-------+-------+-------+------+-------+-----+--------+--------+-----+-------+-------+-------+-------+-------+--------+------+------+-------+--------------+-----+--------+
|  7.0|2016.0|   4.0| 254.0| 276.0|    ATL|20551.0|    1.0|     AL|   null|  25.0|    3.0|  1.0|20130811|     SEO| null|      G|   null|      Y|   null| 1991.0|     D/S|     M|  null|   null|  3.73679633E9|00296|      F1|
| 15.0|2016.0|   4.0| 101.0| 101.0|    WAS|20545.0|    1.0|     MI|20691.0|  55.0|    2.0|  1.0|20160401|    nul

In [18]:
## Clean the temperature data
len(df_temp['City'].unique())

3448

In [23]:
df_temp['City'].isnull().sum()
## We have no null values in the City column which is good

0

In [24]:
df_temp.columns

Index(['dt', 'AverageTemperature', 'AverageTemperatureUncertainty', 'City',
       'Country', 'Latitude', 'Longitude'],
      dtype='object')

In [39]:
def clean_temp (data):
    df_temp_spark = spark.read.format("csv").option("header", "true").load(temp_data)
    df_temp_spark = df_temp_spark.filter(df_temp_spark.AverageTemperature != 'NaN')
    df_temp_spark = df_temp_spark.dropDuplicates(['City', 'Country'])
    df_temp_spark = df_temp_spark.withColumn("i94port", conv_city_abbr(df_temp_spark.City))
    df_temp_spark = df_temp_spark.filter(df_temp_spark.i94port != 'null')
    return df_temp_spark

In [29]:
## There are plenty of rows with Nan values and we would have to filter these rows. Using spark engine to filter these rows would be faster
df_temp['AverageTemperature'].isnull().sum()

364130

In [30]:
df_temp_spark = df_temp_spark.filter(df_temp_spark.AverageTemperature != 'NaN')

In [38]:
df_temp_spark = df_temp_spark.dropDuplicates(['City', 'Country'])

In [33]:
len(df_temp)

8599212

In [37]:
## Have to remove the duplicates rows and this shows us that the dataframe reduces to 3490
len(df_temp.drop_duplicates(subset = ['City','Country']))

3490

In [31]:
## Adding the function to add cities abbreivations as an additional column
from pyspark.sql.functions import udf
@udf()
def conv_city_abbr(city):
    '''
    Input: City names from temp
    Output: Corresponding i94_port from the dictionary
    '''
    
    for key in i94_port:
        if city.lower() in i94_port[key][0].lower():
            return key

In [56]:
df_temp_spark = df_temp_spark.withColumn("i94port", conv_city_abbr(df_temp_spark.City))

In [58]:
df_temp_spark = df_temp_spark.filter(df_temp_spark.i94port != 'null')
df_temp_spark.show()

+----------+------------------+-----------------------------+--------+--------------+--------+---------+-------+
|        dt|AverageTemperature|AverageTemperatureUncertainty|    City|       Country|Latitude|Longitude|i94port|
+----------+------------------+-----------------------------+--------+--------------+--------+---------+-------+
|1743-11-01|             8.758|                        1.886|Aberdeen|United Kingdom|  57.05N|    1.48W|    ABE|
|1744-04-01|6.0699999999999985|           2.9339999999999997|Aberdeen|United Kingdom|  57.05N|    1.48W|    ABE|
|1744-05-01|             7.751|                        1.494|Aberdeen|United Kingdom|  57.05N|    1.48W|    ABE|
|1744-06-01|             10.62|                        1.574|Aberdeen|United Kingdom|  57.05N|    1.48W|    ABE|
|1744-07-01|             12.35|                        1.591|Aberdeen|United Kingdom|  57.05N|    1.48W|    ABE|
|1744-09-01|            11.224|           1.6059999999999999|Aberdeen|United Kingdom|  57.05N|  

### Step 3: Define the Data Model
#### 3.1 Conceptual Data Model
Map out the conceptual data model and explain why you chose that model
The idea behind the data model is to create a fact table and dimensions table. The fact table would have all the important columns from the immigration data and the dimensions table
would be subgroup from the immigration and the temperature data 

#### Fact Table:
Columns
- i94port = code for the USA ports (abbreivation)
- AverageTemperature = average temperature destination city
- i94cit = code for origin city

#### Dimenstion Table: Immigration
- i94yr = year column 
- i94mon = month column
- i94mode = single digit travel code
- arrdate = arrival date
- i94visa = reason for immigration
- i94port = code for the USA ports (This would be the joining key)
- depdate = Departure date

#### Dimension Table: Temperature
- City = city name
- Country = Name of the country 
- Latitude = Latitude
- Longitude = Longitude
- i94port = code for the USA ports (This would be the joining key)

#### 3.2 Mapping Out Data Pipelines
- Clean the I94 immigration data to remove the Null values from the City and residential address columns
- Clean the temperature data to remove the null values from Cities and Average Temperature columns
- Create fact table by joining immigration and temperature data and selecting the relvant columns
- Create immigration dimension table by selecting the relevant columns
- Create temperature dimenstion table by selecting the relevant columns


### Step 4: Run Pipelines to Model the Data 
#### 4.1 Create the data model
Build the data pipelines to create the data model.


In [40]:
## Cleaning the immigration data
immi_data = '../../data/18-83510-I94-Data-2016/i94_apr16_sub.sas7bdat'
df_immi = clean_immi(immi_data,i94_port,"i94port")

In [42]:
## Extracting the columns for immigration dimension table
immi_table = df_immi.select(["i94yr", "i94mon", "i94port", "arrdate", "i94mode", "depdate", "i94visa"])

In [44]:
# Cleaning the temperature data
temp_data = '../../data2/GlobalLandTemperaturesByCity.csv'
df_temp = clean_temp(temp_data)

In [45]:
df_temp.show()

+----------+-------------------+-----------------------------+---------+--------------------+--------+---------+-------+
|        dt| AverageTemperature|AverageTemperatureUncertainty|     City|             Country|Latitude|Longitude|i94port|
+----------+-------------------+-----------------------------+---------+--------------------+--------+---------+-------+
|1852-07-01|             15.488|                        1.395|    Perth|           Australia|  31.35S|  114.97E|    PER|
|1828-01-01|             -1.977|                        2.551|  Seattle|       United States|  47.42N|  121.97W|    SEA|
|1743-11-01|              2.767|                        1.905| Hamilton|              Canada|  42.59N|   80.73W|    HAM|
|1849-01-01|  7.399999999999999|                        2.699|  Ontario|       United States|  34.56N|  116.76W|    ONT|
|1821-11-01|              2.322|                        2.375|  Spokane|       United States|  47.42N|  117.24W|    SPO|
|1843-01-01| 18.874000000000002|

In [46]:
df_temp.count()

207

In [47]:
# Extract columns for temperature dimension table
temp_table = df_temp.select(["AverageTemperature", "City", "Country", "Latitude", "Longitude", "i94port"])

In [49]:
temp_table.show()

+-------------------+---------+--------------------+--------+---------+-------+
| AverageTemperature|     City|             Country|Latitude|Longitude|i94port|
+-------------------+---------+--------------------+--------+---------+-------+
|             15.488|    Perth|           Australia|  31.35S|  114.97E|    PER|
|             -1.977|  Seattle|       United States|  47.42N|  121.97W|    SEA|
|              2.767| Hamilton|              Canada|  42.59N|   80.73W|    HAM|
|  7.399999999999999|  Ontario|       United States|  34.56N|  116.76W|    ONT|
|              2.322|  Spokane|       United States|  47.42N|  117.24W|    SPO|
| 18.874000000000002|Abu Dhabi|United Arab Emirates|  24.92N|   54.98E|    MAA|
|             25.229|    Anaco|           Venezuela|   8.84N|   64.05W|    ANA|
|              9.904|      Ica|                Peru|  13.66S|   75.14W|    CHI|
|              9.833|  Nogales|       United States|  31.35N|  111.20W|    NOG|
|  8.129999999999999|  Atlanta|       Un

In [50]:
# Create Temporary Views for creating the fact table
df_immi.createOrReplaceTempView("immigration")
df_temp.createOrReplaceTempView("temp")

In [51]:
# Create the fact table by joining the immigration and temperature views
fact_immi_temp = spark.sql('''
SELECT immigration.i94cit as city,
       immigration.i94port as i94port,
       temp.AverageTemperature as temperature
FROM immigration
JOIN temp ON (immigration.i94port = temp.i94port)
''')

In [52]:
fact_immi_temp.show()

+-----+-------+-----------------+
| city|i94port|      temperature|
+-----+-------+-----------------+
|111.0|    SNA|7.168999999999999|
|114.0|    SNA|7.168999999999999|
|117.0|    SNA|7.168999999999999|
|129.0|    SNA|7.168999999999999|
|575.0|    SNA|7.168999999999999|
|575.0|    SNA|7.168999999999999|
|577.0|    SNA|7.168999999999999|
|577.0|    SNA|7.168999999999999|
|582.0|    SNA|7.168999999999999|
|582.0|    SNA|7.168999999999999|
|582.0|    SNA|7.168999999999999|
|582.0|    SNA|7.168999999999999|
|582.0|    SNA|7.168999999999999|
|582.0|    SNA|7.168999999999999|
|582.0|    SNA|7.168999999999999|
|582.0|    SNA|7.168999999999999|
|582.0|    SNA|7.168999999999999|
|582.0|    SNA|7.168999999999999|
|582.0|    SNA|7.168999999999999|
|582.0|    SNA|7.168999999999999|
+-----+-------+-----------------+
only showing top 20 rows



#### 4.2 Data Quality Checks
Explain the data quality checks you'll perform to ensure the pipeline ran as expected. These could include:
 * Integrity constraints on the relational database (e.g., unique key, data type, etc.)
 * Unit tests for the scripts to ensure they are doing the right thing
 * Source/Count checks to ensure completeness
 
Run Quality Checks

In [53]:
# Perform quality checks here
def quality(df):
    '''
    Input: Spark dataframe
    Output: Print outcome of data quality check
    '''
    
    result = df.count()
    if result == 0:
        print("No records")
    else:
        print("Number of records =  {} ".format(result))
    return 0

In [ ]:
# Perform data quality check
quality(df_immi)
quality(df_temp)

Number of records =  3088544 


#### 4.3 Data dictionary 
Create a data dictionary for your data model. For each field, provide a brief description of what the data is and where it came from. You can include the data dictionary in the notebook or in a separate file.

### Fact Table:
Contains a combination of immigration data and temperature data that has been combined by i94port

Columns
- i94port = code for the USA ports (abbreivation)
- AverageTemperature = average temperature destination city
- i94cit = code for origin city

### Dimension Table: Immigration
- i94yr = year column 
- i94mon = month column
- i94mode = single digit travel code
- arrdate = arrival date
- i94visa = reason for immigration
- i94port = code for the USA ports (This would be the joining key)
- depdate = Departure date

### Dimenstion Table: Temperature
- City = city name
- Country = Name of the country 
- Latitude = Latitude
- Longitude = Longitude
- i94port = code for the USA ports (This would be the joining key)

#### Step 5: Complete Project Write Up
* Clearly state the rationale for the choice of tools and technologies for the project.
    - I used Spark for this project because of its capability to handle large sets of data in different formats (CSV,SAS,txt). The fact table was  created   using Spark SQL to read data from immigration and temperature data and join them using SQL.
 
 
* Propose how often the data should be updated and why.
    - Monthly because the format of the data is saved monthly

* Write a description of how you would approach the problem differently under the following scenarios:
 * The data was increased by 100x.
    - Load into Redshift. It can do the heavy lifting and load data in a larger scale 
 * The data populates a dashboard that must be updated on a daily basis by 7am every day.
    - Using Airflow we can create a trigger to run the pipeline on the previous night. Integrate the job into the airflow and run it every day the previous night
 * The database needed to be accessed by 100+ people.
    - We would need more resources inorder to get a faster experience. By using distributed database and scaling up our resources we can get faster results for the 100+ people
    - 